#추천시스템1

 - 

In [ ]:
pip install pandas

In [ ]:
pip install konlpy

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from scipy.sparse import vstack
from konlpy.tag import Okt

# CSV 파일 로드
data = pd.read_csv('../data/supplements_20250311.csv')  # your_data.csv에 데이터 로드 (가정: 텍스트 컬럼이 '기능성'이라고 가정)

# KoNLPy의 Okt 객체를 사용하여 형태소 분석
okt = Okt()

# 불용어 목록을 텍스트 파일에서 읽어오기
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()
    return stopwords

# 불용어 리스트 로드
stop_words = load_stopwords('../data/stopwords-ko.txt')

# 텍스트 전처리 함수 (형태소 분석 및 불용어 처리)
def preprocess_text(text):
    if pd.isna(text):  # NaN 값이 있을 경우 빈 문자열로 처리
        return ''
    try:
        # 형태소 분석을 통해 명사만 추출
        nouns = okt.nouns(text)
        # 불용어 제거
        filtered_nouns = [word for word in nouns if word not in stop_words]
        return ' '.join(filtered_nouns) if filtered_nouns else ''
    except Exception as e:
        print(f"Error processing text: {text[:30]}... -> {e}")  # 텍스트 앞 30글자만 출력
        return ''

# 텍스트 전처리
data['processed_text'] = data['기능성'].apply(preprocess_text)

# TF-IDF 벡터화 함수
def process_tfidf_batch(batch_data, vectorizer=None):
    if vectorizer is None:
        vectorizer = TfidfVectorizer()  # 불용어 처리 이미 완료된 데이터 사용
        tfidf_matrix = vectorizer.fit_transform(batch_data)  # TF-IDF 매트릭스 생성
    else:
        tfidf_matrix = vectorizer.transform(batch_data)  # 기존 벡터라이저를 사용하여 변환
    return tfidf_matrix, vectorizer

# 배치 단위로 처리하기
batch_size = 5000  # 5000개씩 처리 (배치 크기 조정 가능)
tfidf_results = []

# 첫 번째 배치에서 벡터라이저 학습
first_batch = data['processed_text'][:batch_size]
tfidf_matrix, tfidf_vectorizer = process_tfidf_batch(first_batch)

tfidf_results.append(tfidf_matrix)

# 나머지 배치 처리
for start in range(batch_size, len(data), batch_size):
    end = min(start + batch_size, len(data))
    batch_data = data['processed_text'][start:end]
    
    tfidf_batch, _ = process_tfidf_batch(batch_data, tfidf_vectorizer)
    tfidf_results.append(tfidf_batch)

# 희소 행렬 합치기 (결과 병합)
final_tfidf_matrix = vstack(tfidf_results)  # 모든 배치 결과를 하나의 희소 행렬로 결합

# 결과 저장 (옵션)
joblib.dump(final_tfidf_matrix, 'final_tfidf_matrix.pkl')  # 결과를 파일로 저장 (필요한 경우)

# 결과 확인
print(final_tfidf_matrix.shape)  # 희소 행렬의 형태 출력 (행: 샘플, 열: 단어)


(40459, 482)


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
from scipy.sparse import vstack
from konlpy.tag import Okt
import numpy as np

# CSV 파일 로드
data = pd.read_csv('../data/supplements_20250311.csv')  # 텍스트 컬럼 '기능성' 사용

# KoNLPy의 Okt 객체를 사용하여 형태소 분석
okt = Okt()

# 불용어 목록을 텍스트 파일에서 읽어오기
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()
    return stopwords

# 불용어 리스트 로드
stop_words = load_stopwords('../data/stopwords-ko.txt')

# 텍스트 전처리 함수 (형태소 분석 및 불용어 처리)
def preprocess_text(text):
    if pd.isna(text):  # NaN 값이 있을 경우 빈 문자열로 처리
        return ''
    try:
        # 형태소 분석을 통해 명사만 추출
        nouns = okt.nouns(text)
        # 불용어 제거
        filtered_nouns = [word for word in nouns if word not in stop_words]
        return ' '.join(filtered_nouns) if filtered_nouns else ''
    except Exception as e:
        print(f"Error processing text: {text[:30]}... -> {e}")  # 텍스트 앞 30글자만 출력
        return ''

# 텍스트 전처리
data['processed_text'] = data['기능성'].apply(preprocess_text)

# TF-IDF 벡터화 함수
def process_tfidf_batch(batch_data, vectorizer=None):
    if vectorizer is None:
        vectorizer = TfidfVectorizer()  # 불용어 처리 이미 완료된 데이터 사용
        tfidf_matrix = vectorizer.fit_transform(batch_data)  # TF-IDF 매트릭스 생성
    else:
        tfidf_matrix = vectorizer.transform(batch_data)  # 기존 벡터라이저를 사용하여 변환
    return tfidf_matrix, vectorizer

# 배치 단위로 처리하기
batch_size = 5000  # 5000개씩 처리 (배치 크기 조정 가능)
tfidf_results = []

# 첫 번째 배치에서 벡터라이저 학습
first_batch = data['processed_text'][:batch_size]
tfidf_matrix, tfidf_vectorizer = process_tfidf_batch(first_batch)

tfidf_results.append(tfidf_matrix)

# 나머지 배치 처리
for start in range(batch_size, len(data), batch_size):
    end = min(start + batch_size, len(data))
    batch_data = data['processed_text'][start:end]
    
    tfidf_batch, _ = process_tfidf_batch(batch_data, tfidf_vectorizer)
    tfidf_results.append(tfidf_batch)

# 희소 행렬 합치기 (결과 병합)
final_tfidf_matrix = vstack(tfidf_results)  # 모든 배치 결과를 하나의 희소 행렬로 결합

# 단어 중요도 합산
word_importance = np.array(final_tfidf_matrix.sum(axis=0)).flatten()

# 단어와 중요도 (TF-IDF 점수 합) 연결
words = list(tfidf_vectorizer.get_feature_names_out())
word_importance_dict = dict(zip(words, word_importance))

# 중요도가 높은 단어들 상위 100개 추출
sorted_words = sorted(word_importance_dict.items(), key=lambda x: x[1], reverse=True)
top_100_words = sorted_words[:100]

# 결과 출력
print("상위 100개 단어:")
for word, importance in top_100_words:
    print(f"{word}: {importance}")
 

상위 100개 단어:
비타민: 4678.347864210655
에너지: 4200.570580034271
배변: 3854.336257172601
혈액: 3812.3215550227915
항산화: 3793.3290254096873
산소: 3769.231749214203
면역: 3766.880869395086
유산균: 3254.479111933815
감소: 3008.492446754109
기억: 2845.5906389369397
피로: 2783.032539733685
골다공증: 2207.1877303518
단백질: 2200.6113133665554
칼슘: 2130.345497804262
지방: 1930.3185987553309
시스테인: 1521.2363199142403
피부: 1520.6129753008752
체지방: 1513.3719572724335
아연: 1455.6845310617175
프로바이오틱스: 1431.4937970582157
혈행: 1239.0534434972858
근육: 1162.8954368526597
신경: 1131.9009699257895
콜레스테롤: 1083.1251164582902
아미노산: 1076.9195921891505
관절: 1075.3969392964261
지질: 1017.7424233612493
연골: 1007.6127497282199
중성: 987.1897055728577
엽산: 909.2835226819092
점막: 806.8639137065445
홍삼: 784.4734572421034
상피세포: 758.3461828948556
치아: 703.5706923638827
경관: 604.2722604279988
태아: 604.0387515341207
운반: 603.1206662895527
셀렌: 597.907903679144
혈당: 581.2977033796352
나이아신: 562.8007889046536
식후: 556.6049067229468
상승: 550.3032330158466
판토텐산: 538.3353640536776
보